# Introduction to the accidents dataset
In this Notebook, you'll take your first look at the accidents dataset. We'll only be looking at the accidents for 2012, as that's all that will fit into the 32-bit MongoDB instance running on the virtual machine. 

This dataset contains details of every road traffic accident in Britain in 2012. The data is anonymised, but the date, time, and location of the accidents is recorded, along with the number and severity of casualties.

It will reinforce some of the ways of querying data with MongoDB and recap some ways of using *pandas* to analyse the data. 

In [ ]:
# Import the required libraries

import pymongo
import datetime
import collections

import pandas as pd
import scipy.stats

In [ ]:
# Open a connection to the Mongo server, open the accidents database and name the collections of accidents and labels
client = pymongo.MongoClient('mongodb://localhost:27351/')

db = client.accidents
accidents = db.accidents
labels = db.labels

In [ ]:
# Load the expanded names of keys and human-readable codes into memory
# Don't bother reading this cell yet: we'll use it in a bit.

expanded_name = collections.defaultdict(str)
for e in labels.find({'expanded': {"$exists": True}}):
    expanded_name[e['label']] = e['expanded']
    
label_of = collections.defaultdict(str)
for l in labels.find({'codes': {"$exists": True}}):
    for c in l['codes']:
        try:
            label_of[l['label'], int(c)] = l['codes'][c]
        except ValueError: 
            label_of[l['label'], c] = l['codes'][c]

# A note on speed limits
The speed limit data in this dataset shows the speed limit of the road at the location of the accident. It says nothing about the speed of any particular vehicle, so you can't use this data to infer anything about whether speeding causes more accidents. However, it's reasonable to assume that vehicles will often be going faster in a 60mph zone than in a 30mph zone.

# Getting a single document
`find_one()` is the basic method for returning a single document from a collection. With no arguments, it just returns the first document it finds (chosen arbitrarily by Mongo).

In [ ]:
accidents.find_one()

That's quite a bit of data, and it's not that useful to select an arbitrary accident report. Let's pull out an accident that happened in a 70 mph zone. We do that by giving a document that must match one in the database before it's returned.

In [ ]:
accidents.find_one({'Speed_limit': 70})

If we give more than one key-value pair in the query document, the returned document must match all of them (an AND). For instance, to find an accident in a 30mph zone that involved two vehicles and one casualty, we specify that information in the query document:

In [ ]:
accidents.find_one({'Speed_limit': 30, 'Number_of_Vehicles': 2, 'Number_of_Casualties': 1})

Recall that we can limit the key-value pairs returned by specifying the second argument to `find_one()`. 

This combines *selection* (the speed limit, one casualty, and two vehicles) and *projection* (only retrieving some parts of the document).

In [ ]:
accidents.find_one({'Speed_limit': 30, 'Number_of_Casualties': 1, 'Number_of_Vehicles': 2}, 
                   ['Accident_Index', 'Speed_limit', 'Number_of_Casualties', 'Number_of_Vehicles'])

# Finding multiple documents
If we want to find more than one document at a time, we use the imaginatively named `find()` function.

One useful thing the cursor can do is tell us how many documents match.

In [ ]:
accidents.find({'Speed_limit': 30, 'Number_of_Casualties': 1, 'Number_of_Vehicles': 2}).count()

With this dataset, the `limit` query keyword is extremely useful when exploring, as it stops us being overwhelmed by data. Let's create a small DataFrame to pick out a few attributes of a few accidents.

Recall that the output of `find()` is an iterator of `dict`s. If we convert the iterator to a `list`, we can create a DataFrame directly.

In [ ]:
pd.DataFrame(list(accidents.find({'Speed_limit': 30, 'Number_of_Casualties': 1, 'Number_of_Vehicles': 2}, 
                                 ['Accident_Index', 'Accident_Severity', 'Road_Type','Weather_Conditions'],
                                 limit=10)))

### Activity 1
How many accidents were there at less than 30mph?

Create a DataFrame that holds the accident index, number of vehicles, and number of casualties for each accident, where the speed limit is less than 30mph. 

(You may find it useful to revise 'Query criteria' in _MongoDB: The Definitive Guide_.)

The solution is in the [`14.2solutions`](14.2solutions.ipynb) Notebook.

In [ ]:
#Insert your solution here

# Expanding the codes
A lot of the information in this dataset is recorded as codes rather than human-readable labels. The `labels` collection contains the labels for each code. 

Here are some examples for both looking up individual values and seeing what labels and codes there are. Treat these as recipes for your own explorations.

The `labels` collection contains the human-readable labels for all the codes in in the accident descriptions. We can ask for the labels and codes for `Road_Type`:

In [ ]:
labels.find_one({'label': 'Road_Type'})

We can ask for the label of on particular type of road.

In [ ]:
labels.find_one({'label': 'Road_Type'})['codes']['6']

We can ask for all the labels:

In [ ]:
road_types = []
road_code_labels = labels.find_one({'label': 'Road_Type'})['codes']
for code in road_code_labels:
    road_types.append(road_code_labels[code])
road_types

But, rather than doing lots of `append`s in a `for` loop, there's a more idiomatic Python approach: a _list comprehension_.

In [ ]:
[road_code_labels[code] for code in road_code_labels]

To make things easier, the piece of code at the top of the notebook loads all the labels and codes into a `dict` called `label_of`. You can find the label of a code for a key by looking in there. The dict's keys are 2-tuples; the first element is the accident document's key we're looking up and the second is the code. The value is the human-readable label for that code. 

In [ ]:
label_of[('Road_Type', 6)]

Note that we dn't need the brackets around the 2-tuple when Python knows that's what we mean, which can make the code a little more readable.

In [ ]:
label_of['Road_Type', 6]

You can find all the codes and labels for a key by walking over the `dict`:

In [ ]:
road_types = []
for (key, code) in label_of:
    if key == 'Road_Type':
        road_types.append(label_of[key, code])
road_types

or, more idiomatically, with a list comprehension. Note how the important parts of the cell above, the `for` and `if`, are still in the comprehension form.

In [ ]:
[label_of[key, code] 
 for (key, code) in label_of 
 if key == 'Road_Type']

You can return the code with the label by wrapping them in a `tuple`. Again, note that we don't always need brackets aroun the tuple. The brackets are needed in the "return" position, but not in the `for` statement.

In [ ]:
[(code, label_of[key, code]) 
 for key, code in label_of 
 if key == 'Road_Type']

### Activity 2
Find and display the document in the `labels` collection that stores details of weather conditions. 

Use the `label_of` dict to find all the labels for `Weather_Conditions`.

Use the `label_of` dict in a list comprehension to find all the labels for `Weather_Conditions` that refer to wind. Note that you can include more than one `if` clause in a list comprehension: just put them one after the other, like this:

    ```[item for item in thing if condition1 if condition2]```
    
It will only return items where both conditions are true.

The solution is in the [`14.2solutions`](14.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here

Some keys in the documents are themselves cryptic, and the human-readable expanions of them are available in the `labels` collection. `expanded_name` dict gives their full names. You'll see these more in Part 15 when we look at the road census data.

In [ ]:
# What does 'FdHGVR4' mean?
labels.find_one({'label': 'FdHGVR4'})

The `expanded_name` dict also gives their full names. You'll see these more in Part 15 when we look at the road census data.

In [ ]:
# What does 'FdHGVR4' mean?
expanded_name['FdHGVR4']

We can see which keys have labels for codes:

## Back to queries
Armed with this information, we can repeat the query earlier and list the accident indexes together with the human-readable data.

### Activity 3
The earlier query is below, but it prints the results rather than putting them in a DataFrame.

In [ ]:
for a in accidents.find({'Speed_limit': 30, 'Number_of_Casualties': 1, 'Number_of_Vehicles': 2}, 
                                 ['Accident_Index', 'Accident_Severity', 'Road_Type','Weather_Conditions'],
                                 limit=10):
    print(a['Accident_Index'],  
          a['Accident_Severity'],
          a['Road_Type'], 
          a['Weather_Conditions'],
          a['_id'])

Repeat this query, but print the labels of the accident severity, road type, and weather conditions (e.g. 
```
201201BS70001 Slight Single carriageway Fine no high winds
```
)

The solution is in the [`14.2solutions`](14.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here

In this case, storing the looked-up values in the DataFrame is tricker, as we can't just use the dicts that come out of the `find()`. We have to do some processing and manually build a new dict for each document found.

In [ ]:
pd.DataFrame([
        {'Accident_Index': a['Accident_Index'],
          'Accident_Severity': label_of['Accident_Severity', a['Accident_Severity']],
          'Road_Type': label_of['Road_Type', a['Road_Type']], 
          'Weather_Conditions': label_of['Weather_Conditions', a['Weather_Conditions']]}
        for a in accidents.find({'Speed_limit': 30, 
                                 'Number_of_Casualties': 1, 'Number_of_Vehicles': 2}, limit=10)])                  

# Investigating the accident data
## Severities at one speed
We can now start to investigate the data. What are the proportions of accident severities in 30mph zones?

Note that the basic `find()` just returns documents: it can't do any aggregation and doesn't have anything like SQL's `GROUP BY` clause. In Part 15, we'll look at Mongo's aggregation pipelines, where we can get Mongo to do this kind of processing for us.

First, we'll collect the summary numbers ...

In [ ]:
# Build a DataFrame, one row for each accident
severities_30_unrolled_df = pd.DataFrame(list(accidents.find({'Speed_limit': 30}, ['Accident_Severity'])))

# Count the number of each severity
severities_30_ss = severities_30_unrolled_df['Accident_Severity'].value_counts()
severities_30_ss

...and plot them.

In [ ]:
severities_30_ss.sort_index().plot(kind='bar')

...or, using the `label_of` dict to find the human-readable labels:

In [ ]:
severities_30_ss.sort_index(inplace=True)
severities_30_ss.index = [label_of['Accident_Severity', r] for r in severities_30_ss.index]
severities_30_ss

In [ ]:
severities_30_ss.plot(kind='bar')

### Activity 4
What are the numbers of accidents at each severity in 60mph zones? Show your results as a bar chart.

The solution is in the [`14.2solutions`](14.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

## Severities across speeds
It's a bit tedious doing one speed at a time. Let's summarise all the data and put it in a *pandas* DataFrame so we can see it all together. 

Let's start by building a DataFrame with one row for each accident. We can then count the number of rows at each speed/severity combination with a _pandas_ `crosstab`. 

In [ ]:
# Build a DataFrame, one row for each accident
severity_by_speed_unrolled_df = pd.DataFrame(list(accidents.find({}, ['Speed_limit', 'Accident_Severity'])))

# Count the number of each severity
severity_by_speed_df = pd.crosstab(severity_by_speed_unrolled_df['Speed_limit'], 
                                   severity_by_speed_unrolled_df['Accident_Severity'])
severity_by_speed_df

We can make the DataFrame more informative by relabelling the columns.

In [ ]:
# Relabel the columns
severity_by_speed_df.columns = [label_of['Accident_Severity', s] for s in severity_by_speed_df.columns]
severity_by_speed_df

In [ ]:
severity_by_speed_df.plot(kind='bar')

As an aside, it's sometimes useful to move the graph legend off to the side. Use this reformulation of the plotting to do so.

In [ ]:
ax = severity_by_speed_df.plot(kind='bar')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

As you can see, there are a lot more accidents in 30mph zones than at any other speed limit. We'll have to account for that when we do some more detailed analysis later.

### Activity 5
What are the proportions of accident severities by junction type (roundabout, crossroads, etc)?

This is a more in-depth activity than before, so I've broken it down into steps.

* What are the different types of junction? You'll need to look at the `label_of` dict to find the right key
* What different severities happen at each junction type?

Comment on your results.

The solution is in the [`14.2solutions`](14.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here. 
# You will likely need several cells to complete this activity

# Looking at subdocuments
Each `accident` document contains two lists of subdocuments: one for each casualty, and one for each vehicle involved in the accident. How can we analyse those subdocuments?

Let's look at the distribution of ages across all accidents, but restrict our investigation to just the area around Milton Keynes, the home of The Open University. 

Is there any metadata about the accidents that relates to Milton Keynes?

In [ ]:
# All the labels that are to do with 'Milton'.
[(k, c, label_of[k, c]) for k, c in label_of if 'Milton' in label_of[k, c]]

While the district authority looks like the one that would have most meaning for people, the "highway" district authority is the one used by the rest of UK government. 

The casualty and vehicle subdocuments are held in arrays/lists, as there are different numbers of them for each accidents. This means we need to iterate over the casualty list within each accident. 

One way to get to the ages is by explicit two-level iteration, building up a list of age bands.

In [ ]:
age_bands_of_casualties_list = []
for a in accidents.find({'Local_Authority_(Highway)': 'E06000042'}):
    for c in a['Casualties']:
        age_bands_of_casualties_list += [c['Age_Band_of_Casualty']]
        
aboc_ss = pd.Series(age_bands_of_casualties_list)
aboc_ss.count(), aboc_ss.head(10)

We can now use standard _pandas_ operations on the series, such as counting how many casualties there are in each age band.

In [ ]:
age_bands_of_casualties_ss = aboc_ss.value_counts()
age_bands_of_casualties_ss

We can do the same thing with a comprehension, to avoid all the list appending. Note that there are two iterators in the comprehension, and they are in the same order as the nested `for` statements above.

In [ ]:
aboc_ss = pd.Series(c['Age_Band_of_Casualty']
                    for a in accidents.find({'Local_Authority_(Highway)': 'E06000042'})
                    for c in a['Casualties'])

age_bands_of_casualties_ss = aboc_ss.value_counts()
age_bands_of_casualties_ss

Finally, we can skip the intermediate `aboc` series by combining the Series creation and counting stages.

In [ ]:
age_bands_of_casualties_ss = pd.Series(c['Age_Band_of_Casualty']
                    for a in accidents.find({'Local_Authority_(Highway)': 'E06000042'})
                    for c in a['Casualties']).value_counts()
age_bands_of_casualties_ss

In [ ]:
age_bands_of_casualties_ss.sort_index(inplace=True)
age_bands_of_casualties_ss

In [ ]:
pd.Series(age_bands_of_casualties_ss).plot(kind='bar')

That's similar to what we'd expect, with fewer young and old casualties as there will be fewer of them on the road.

### Activity 6
We can also ask if different age people respond to injuries differently. In other words, are old people more frail and so therefore more likely to be seriously or fatally injured?

Following the examples above, let's build a DataFrame that counts the number of casualties for each age band / severity combination, and use the human-readable labels for the data rather than the codes.

You should end up with a `DataFrame` called `severity_by_age_df` that looks like this:


Severity | Fatal | Serious | Slight
-|-|-|-
Unknown | 0 | 0 | 8
0 - 5 | 0 | 3 | 20
6 - 10 | 0 | 1 | 28
11 - 15 | 0 | 3 | 34
16 - 20 | 1 | 11 | 115
21 - 25 | 1 | 16 | 133
26 - 35 | 2 | 15 | 219
36 - 45 | 3 | 12 | 168
46 - 55 | 0 | 10 | 125
56 - 65 | 1 | 7 | 63
66 - 75 | 0 | 9 | 28
Over 75 | 2 | 2 | 20

The solution is in the [`14.2solutions`](14.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

Now you have the age data, can we plot it to see what it means?

In [ ]:
severity_by_age_df.plot()

The number of `slight` casualties is (thankfully) much larger than the other severities, so it's difficult to see if the proportions in different age bands remain similar. We can try plotting the data in separate graphs, mindful that the y-axis scales will be different in each graph. (It will show if there's a different pattern across severities.)

In [ ]:
severity_by_age_df.plot(subplots='True')

…or on one graph, using secondary y axis for non-slight casualties.

In [ ]:
severity_by_age_df['Slight'].plot()
severity_by_age_df['Serious'].plot(secondary_y=True)
severity_by_age_df['Fatal'].plot(secondary_y=True)

There doesn't seem to be a vast difference in the proportion of casualties. (The number of deaths is too low to draw any firm conclusions from.)

## Size of accidents: comparing numbers of vehicles and casualties
Some accidents are 'big' and some are 'small'. There are two obvious ways of describing such accidents: the number of casualties or the number of vehicles. We'd expect these two measures to be correlated. Let's find out if that's the case.

First, we'll read the sizes of accidents from the database and count them, as before

In [ ]:
# Build a DataFrame, one row for each accident
cas_veh_unrolled_df = pd.DataFrame(list(accidents.find({}, ['Number_of_Casualties', 'Number_of_Vehicles'])))

# Count the number of each severity
cas_veh_df = pd.crosstab(cas_veh_unrolled_df['Number_of_Casualties'], 
                                      cas_veh_unrolled_df['Number_of_Vehicles'])
cas_veh_df

We want to plot this data as a scatter plot, but that function requires a sequences of casualty numbers, vehicle numbers, and counts of accidents for each pair. To convert this data into three sequences, we'll use the `stack()` and `reset_index()` methods you saw earlier.

In [ ]:
cas_veh_long_df = cas_veh_df.stack().reset_index()
cas_veh_long_df

We can now plot the number of accidents at each casualty/vehicle combination on a *scatter chart*, a scatter plot that additionally uses the _size_ of each point to communicate information about a dimension of the data. In this case, we'll scale the point size by the number of accidents of that 'size'.

In [ ]:
plt.scatter(cas_veh_long_df['Number_of_Casualties'], 
            cas_veh_long_df['Number_of_Vehicles'],
            s=np.sqrt(cas_veh_long_df[0])*1.5,
            alpha=0.5
            )
plt.xlabel('Number of casualties')
plt.ylabel('Number of vehicles')
plt.show()

There are a number of outliers, so let's remove them to see the rest of the points more easily. Limiting the data to 20 vehicles and 10 casualties looks reasonable.

In [ ]:
small_cvcl_df = cas_veh_long_df[(cas_veh_long_df['Number_of_Casualties'] <= 10) & 
                                (cas_veh_long_df['Number_of_Vehicles'] <= 20)]

plt.scatter(small_cvcl_df['Number_of_Casualties'], 
            small_cvcl_df['Number_of_Vehicles'],
            s=np.sqrt(small_cvcl_df[0])*3,
            alpha=0.5
            )
plt.xlabel('Number of casualties')
plt.ylabel('Number of vehicles')
plt.show()

Now we have some data plotted, can we say anything about the correlation of the two different metrics for 'size' of accident? From first impressions, there's not much correlation between the two measures: most of the accidents are small in both number of casualties and vehicles, and tend to be in the lower-left corner.

The obvious first step is to find the line of best fit, and then plot it.

Unfortunately, finding a regression line requires the data to be 'unrolled', with one row for each accident. The `cas_veh_unrolled_df` has this information, but we need to pull out the relevant columns.

In [ ]:
cas_veh_unrolled_df.head()

In [ ]:
regressionline = scipy.stats.linregress(cas_veh_unrolled_df['Number_of_Casualties'],
                                       cas_veh_unrolled_df['Number_of_Vehicles'])
# The regression line is of the form y = m x + b
m = regressionline[0]
b = regressionline[1]
(m, b)

Now, plot the regression line on top of the scatter plot

In [ ]:
plt.scatter(small_cvcl_df['Number_of_Casualties'], 
            small_cvcl_df['Number_of_Vehicles'],
            s=np.sqrt(small_cvcl_df[0])*3,
            alpha=0.5
            )
plt.xlabel('Number of casualties')
plt.ylabel('Number of vehicles')

x = np.linspace(0, 30, 20)
plt.plot(x, m*x + b)

plt.show()

There is a line of best fit, but it doesn't seem to describe the data very well. In the next Notebook, we'll look at how to get a better handle on the this correlation.

### Activity 7

How does the number of vehicles in an accident vary with the speed limit?

Find the results, find the line of best fit, then plot the data and line of best fit on a scatter graph.

The solution is in the [`14.2solutions`](14.2solutions.ipynb) Notebook.

In [ ]:
# Insert your solution here.

## What next?
If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, move on to `14.3 Using statistical tests`.